In [6]:
import tensorflow as tf

import numpy as np
import os
import time

In [7]:
# Read, then decode for py2 compat.
text = open('/content/sample_data/shakespeare-alls-11.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 135197 characters


In [8]:
# Take a look at the first 250 characters in text
print(text[:350])

	ALL'S WELL THAT ENDS WELL

	DRAMATIS PERSONAE

KING OF FRANCE	(KING:)

DUKE OF FLORENCE	(DUKE:)

BERTRAM	Count of Rousillon.

LAFEU	an old lord.

PAROLLES	a follower of Bertram.

Steward	|
	|  servants to the Countess of Rousillon.
Clown	|

	A Page. (Page:)

COUNTESS OF
ROUSILLON	mother to Bertram. (COUNTESS:)

HELENA	a gentlewoman protected by th


In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

67 unique characters


## Process the text

In [10]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [13]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[41, 42, 43, 44, 45, 46, 47], [64, 65, 66]]>

In [14]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [18]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [21]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

In [22]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(135197,), dtype=int64, numpy=array([ 1, 15, 26, ..., 60, 40,  2])>

In [23]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [25]:
for ids in ids_dataset.take(15):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

	
A
L
L
'
S
 
W
E
L
L
 
T
H
A


In [26]:
seq_length = 100


In [27]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\t' b'A' b'L' b'L' b"'" b'S' b' ' b'W' b'E' b'L' b'L' b' ' b'T' b'H'
 b'A' b'T' b' ' b'E' b'N' b'D' b'S' b' ' b'W' b'E' b'L' b'L' b'\n' b'\n'
 b'\t' b'D' b'R' b'A' b'M' b'A' b'T' b'I' b'S' b' ' b'P' b'E' b'R' b'S'
 b'O' b'N' b'A' b'E' b'\n' b'\n' b'K' b'I' b'N' b'G' b' ' b'O' b'F' b' '
 b'F' b'R' b'A' b'N' b'C' b'E' b'\t' b'(' b'K' b'I' b'N' b'G' b':' b')'
 b'\n' b'\n' b'D' b'U' b'K' b'E' b' ' b'O' b'F' b' ' b'F' b'L' b'O' b'R'
 b'E' b'N' b'C' b'E' b'\t' b'(' b'D' b'U' b'K' b'E' b':' b')' b'\n' b'\n'
 b'B' b'E' b'R'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'This is the 100th Etext file presented by Project Gutenberg, and\r\nis presented in cooperation with Wo'
b'rld Library, Inc., from their\r\nLibrary of the Future and Shakespeare CDROMS.  Project Gutenberg\r\nofte'
b'n releases Etexts that are NOT placed in the Public Domain!!\r\n\r\nShakespeare\r\n\r\n*This Etext has certai'
b'n copyright implications you should read!*\r\n\r\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILL'
b'IAM\r\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\r\nPROVIDED BY PROJECT GUTENBERG'


In [28]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [29]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [30]:
dataset = sequences.map(split_input_target)

In [31]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"\tALL'S WELL THAT ENDS WELL\n\n\tDRAMATIS PERSONAE\n\nKING OF FRANCE\t(KING:)\n\nDUKE OF FLORENCE\t(DUKE:)\n\nBE"
Target: b"ALL'S WELL THAT ENDS WELL\n\n\tDRAMATIS PERSONAE\n\nKING OF FRANCE\t(KING:)\n\nDUKE OF FLORENCE\t(DUKE:)\n\nBER"


In [32]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [33]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [34]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [35]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 68) # (batch_size, sequence_length, vocab_size)


In [37]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  17408     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  69700     
                                                                 
Total params: 4,025,412
Trainable params: 4,025,412
Non-trainable params: 0
_________________________________________________________________


In [38]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [39]:
sampled_indices

array([60, 15, 10, 59, 28, 46, 59, 36, 67, 30, 40, 63, 42, 49, 56,  7, 63,
       25, 63, 36, 50, 51, 43, 22, 57, 59, 19, 25, 44,  1, 47, 25, 23, 67,
       36, 12, 10, 47, 40,  1, 61, 36, 14,  7, 30, 53, 57, 51, 38, 59,  6,
       48, 22, 10, 12, 30, 60, 36, 24,  3, 38,  7,  8, 50, 33, 48, 43, 39,
       23, 28, 57, 48, 21, 30, 35, 49,  0,  3, 47, 45, 46, 55, 21, 49, 55,
       52,  5, 21, 18, 11, 41, 60,  9, 22, 37,  1, 41, 18, 39, 39])

In [40]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"elves\n\tinto seeming knowledge, when we should submit\n\tourselves to an unknown fear.\n\nPAROLLES\tWhy, '"

Next Char Predictions:
 b"tA-sNfsV|P]wbip(wKwVjkcHqsEKd\tgKI|V:-g]\tuV?(PmqkYs'hH-:PtVJ Y()jShc[INqhGPUi[UNK] gefoGiol&GD.at,HW\taD[["


## Train the model

### Attach an optimizer, and a loss function

In [41]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [42]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 68)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.219366, shape=(), dtype=float32)


In [43]:
tf.exp(example_batch_mean_loss).numpy()

67.99037

In [44]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [45]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [46]:
EPOCHS = 20

In [47]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
20/20 [==============================] - 5s 95ms/step - loss: 4.1745
Epoch 2/20
20/20 [==============================] - 1s 63ms/step - loss: 3.2629
Epoch 3/20
20/20 [==============================] - 2s 71ms/step - loss: 2.8603
Epoch 4/20
20/20 [==============================] - 1s 58ms/step - loss: 2.5627
Epoch 5/20
20/20 [==============================] - 1s 58ms/step - loss: 2.4248
Epoch 6/20
20/20 [==============================] - 1s 58ms/step - loss: 2.3312
Epoch 7/20
20/20 [==============================] - 1s 64ms/step - loss: 2.2509
Epoch 8/20
20/20 [==============================] - 1s 64ms/step - loss: 2.1695
Epoch 9/20
20/20 [==============================] - 1s 57ms/step - loss: 2.0934
Epoch 10/20
20/20 [==============================] - 1s 58ms/step - loss: 2.0201
Epoch 11/20
20/20 [==============================] - 1s 58ms/step - loss: 1.9525
Epoch 12/20
20/20 [==============================] - 1s 58ms/step - loss: 1.8907
Epoch 13/20
20/20 [==================

## Generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
King Lenno Sicancx, both. Come hither, William:
    A staff of gold and mystery; but here, ha!
    Please your lordship the tree, my friend as once;
    The eye's acquainting it a-door's orchar!
    Do not touch thee thy daughter, that  
    To come to both the heavenly perjection
    To our ording masks and die; d, arm'd upon
    The Lady of Something. It doth amign
    She's all your damp's slander not a dream's.
  PORTIA.  [Speaking off BETHUR'S palace

Enter PROSPERO and CONSTANCE

  JEPEIUS. Solecham will Behend will mend no more spiritual,  
    As hate of mine unwelcomen. There's a
    very virtuous; not with careful eternal, so on me.  
    Bid them to vistues whisper hurtless.
  DEMETRIUS. Why look you 'my sister? Kill your eyes?
    O undisgrater sorrow still away!
  BEROWNE.                 A long liver's is a green-sweeper-
    I'll look tarn'd crack. Some sorrow seem!
    Yet I should quickly soon return awhile.
    She is the very argument of death 

______________

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\r\n    How now, my lords! Now but be horroless,\r\n    Pluck pride on't.  [ Exeunt SOLDIERS at, GAUMIO'S pletter]\r\n  HERMIONE. Sir, in such a kind of bees,\r\n    With all the self-same thousand as my means; and shall we  \r\n    When enter'd melting many Worts is quit\r\n    As if that King of France, forror near-\r\n    Now here your high requests more; but since\r\n    I will be sent from the substrich other swears!\r\n    All the next nest o'erlook the breath of jealous,\r\n    Clipp'ring wark, with all the very Moor,\r\n    Attend him-\r\n\r\n                        Enter PARIS\r\n\r\n  VIOLA. Prevent, I thank thee. Here comes your son,\r\n    And will draw her of him.\r\n  CLOWN. [Aside] The noblest, as of a stanger, show of hers;\r\n    These urging meal, now is the better,\r\n    Out, aforeour of the sky.             Exeunt all but PONTAS USEY\r\n  ULYSSES. How dost thou understand A blossom? How didst thou not hear my soul and said\r\n    Do which makes 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO: Paris.

               Enter for Gentleman.

  Cap. Do, thou goes, pyramise!

           Ente


ROMEO:
    My sovereign life, death's tyrannous meet,
    Two lambs our people did fall in their caps
    And something do in my impress, you were come
    Of sudden voices, who, hoping loads,
    Both his truth hath the world again saw stray.
    Let me endure him, if you must dit-
    To give me less enright.
  POLIXENES. From favour!
  ULYSSES. Had she to cuck'd this paper?
  ARIEL. Marry, give them I do. Ho, understand me go,
    Take no husable former-edge of sweet  intents;
    With Jove of her best sorrow'd recepts alone;
    The one with boggans beats the mercy of our horse,
    Who heard of curses speak 'good Rome,
    On him affice; but why has a post
    Are in thy mother's body elder court?
    Clear are my birth, and his behaviour tears?
    This winds do stand with outstarp are.
    Suppos'd this infant from our former ingrate:
    Proudly and I have proved breath with wrinkles,
    And see our trumpet was, unkind to death.
    So should I ever did the dre 